# 1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
pltparams = {'legend.fontsize': 16, 'axes.labelsize': 20, 'axes.titlesize': 20,
             'xtick.labelsize': 12, 'ytick.labelsize':12, 'figure.figsize': (7.5, 7.5),}
plt.rcParams.update(pltparams)
def problem1(cols,taus):
    from datetime import datetime
    from pandas_datareader import data as pdr
    from pandas_datareader import wb as pwb
    # Logarithmic return of price time series
    def logreturn(Pt,tau=1):
        return np.log(Pt[tau:]) - np.log(Pt[0:-tau]) # Eq.(J2) : G_tau(t) = log(S(t+tau)) - log(S(t)) 
    # Normalize data to have unit variance (<(x - <x>)^2> = 1)
    def normalized(data):
        return (data/np.sqrt(np.var(data)))
    # Add G_tau to data frame object
    def computeReturn(data, name, tau):
        data[name]=pd.Series(normalized(logreturn(data['Adj Close'].values, tau)),index=data.index[:-tau])
        
    # define time interval
    end_time   = datetime(2016, 12, 31)
    start_time = datetime(1989, 1, 1)
    apple      = pdr.DataReader('AAPL','yahoo',start_time,end_time)
    for col,tau in zip(cols,taus):
        computeReturn(apple, col, tau)
    return apple
def plot1(axes, data, cols, labels, colors):
    [ax,bx] = axes
    for col,lbl,clr in zip(cols,labels,colors):
        data[col].plot(ax=ax,alpha=0.8, color=clr)
        data[col].hist(ax=bx,alpha=0.6, normed=True, bins=40, lw=0, color=clr, label=lbl)
    ax.set_ylim([-5,5])
    bx.set_xlim([-5,5])
    bx.set_ylim([0,0.6])
    bx.legend()
    ax.set_ylabel('Normalized price return $G_n(t)$')
    bx.set_xlabel('Normalized price return $G_n(t)$')
    bx.set_ylabel('Probability Distribution')

In [ ]:
%matplotlib inline
cols,taus = ['G30','G90','G180'], [30,90,180]
apple = problem1(cols, taus)

fig, axes = plt.subplots(figsize=(15.0,7.5), ncols=2)
plot1(axes, apple, cols, [r'$G_{30}$', r'$G_{90}$', r'$G_{180}$'], ['k','r', 'b'])
fig.tight_layout()
fig.savefig('../fig/06/G03.png')
plt.show()

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(figsize=(15.0,7.5), ncols=2)

cols,taus = ['G2','G5','G10'], [2,5,10]
apple = problem1(cols, taus)
plot1(axes, apple, cols, [r'$G_{30}$', r'$G_{90}$', r'$G_{180}$'], ['k','r', 'b'])
fig.tight_layout()
fig.savefig('../fig/06/G01.png')
plt.show()

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(figsize=(15.0,7.5), ncols=2)

cols,taus = ['G5','G10','G30'], [5,10,30]
apple = problem1(cols, taus)
plot1(axes, apple, cols, [r'$G_{30}$', r'$G_{90}$', r'$G_{180}$'], ['k','r', 'b'])
fig.tight_layout()
fig.savefig('../fig/06/G02.png')
plt.show()

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(figsize=(15.0,7.5), ncols=2)

cols,taus = ['G90','G180','G360'], [90,180,360]
apple = problem1(cols, taus)
plot1(axes, apple, cols, [r'$G_{30}$', r'$G_{90}$', r'$G_{180}$'], ['k','r', 'b'])
fig.tight_layout()
fig.savefig('../fig/06/G04.png')
plt.show()

In [ ]:
%matplotlib inline
fig, axes = plt.subplots(figsize=(15.0,7.5), ncols=2)

cols,taus = ['G5','G90','G360'], [5,90,360]
apple = problem1(cols, taus)
plot1(axes, apple, cols, [r'$G_{30}$', r'$G_{90}$', r'$G_{180}$'], ['k','r', 'b'])
fig.tight_layout()
fig.savefig('../fig/06/G05.png')
plt.show()

# 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
pltparams = {'legend.fontsize': 16, 'axes.labelsize': 20, 'axes.titlesize': 20,
             'xtick.labelsize': 12, 'ytick.labelsize':12, 'figure.figsize': (7.5, 7.5),}
plt.rcParams.update(pltparams)
def problem2(cols,taus):
    from datetime import datetime
    from pandas_datareader import data as pdr
    from pandas_datareader import wb as pwb
    # Logarithmic return of price time series
    def logreturn(Pt,tau=1):
        return np.log(Pt[tau:]) - np.log(Pt[0:-tau]) # Eq.(J2) : G_tau(t) = log(S(t+tau)) - log(S(t)) 
    # Normalize data to have unit variance (<(x - <x>)^2> = 1)
    def normalized(data):
        return (data/np.sqrt(np.var(data)))
    # Add G_tau to data frame object
    def computeReturn(data, name, tau):
        data[name]=pd.Series(normalized(logreturn(data['Adj Close'].values, tau)),index=data.index[:-tau])

    # define time interval
    end_time   = datetime(2016, 12, 31)
    start_time = datetime(1989, 1, 1)
    apple      = pdr.DataReader('AAPL','yahoo',start_time,end_time)
    for col,tau in zip(cols,taus):
        computeReturn(apple, col, tau)
    return apple
def plot2(ax, data, cols, labels, colors):
    # compute normalized probability distribution function
    def pdf(data,bins=50):
        hist, edges = np.histogram(data[~np.isnan(data)], bins=bins, density=True) # remove NaNs and compute histogram (returns bar heights and bar edges)
        edges   = (edges[:-1] + edges[1:])/2.0 # get bar centers
        nonzero = hist > 0.0                   # only keep non-zero points 
        return edges[nonzero], hist[nonzero]

    for col,lbl,clr in zip(cols,labels,colors):
        edges, hist = pdf(np.abs(data[col]), bins=20)
        ax.plot(edges, hist, label=lbl, lw=3, color=clr)

    #plot gaussian
    x = np.logspace(-1, 1.2)
    ax.plot(x,np.abs(np.exp(-x**2/2)/np.sqrt(2*np.pi)),lw=6,ls='--',color='gray',alpha=0.8,label=r'Gaussian')
        
    ax.set_ylim(1e-4, 2e0)
    ax.set_xlim(1e-1, 2e1)
    ax.semilogx()
    ax.semilogy()
    ax.legend(loc=3)
    ax.set_xlabel(r'Absolute normalized price return $|G_n|$')
    ax.set_ylabel(r'Probability distribution')

In [ ]:
%matplotlib inline
cols,taus = ['G30','G90','G180'], [30,90,180]
apple = problem2(cols, taus)

fig, ax = plt.subplots(figsize=(7.5,7.5))
plot2(ax, apple, cols, [r'$G_{30}$', r'$G_{90}$', r'$G_{180}$'], ['k','r', 'b'])
fig.tight_layout()
fig.savefig('../fig/06/G12.png')
plt.show()

In [ ]:
%matplotlib inline
cols,taus = ['G2','G5','G10'], [2,5,10]
apple = problem2(cols, taus)

fig, ax = plt.subplots(figsize=(7.5,7.5))
plot2(ax, apple, cols, [r'$G_{30}$', r'$G_{90}$', r'$G_{180}$'], ['k','r', 'b'])
fig.tight_layout()
fig.savefig('../fig/06/G11.png')
plt.show()

In [ ]:
%matplotlib inline
cols,taus = ['G5','G10','G30'], [5,10,30]
apple = problem2(cols, taus)

fig, ax = plt.subplots(figsize=(7.5,7.5))
plot2(ax, apple, cols, [r'$G_{30}$', r'$G_{90}$', r'$G_{180}$'], ['k','r', 'b'])
fig.tight_layout()
fig.savefig('../fig/06/G15.png')
plt.show()

In [ ]:
%matplotlib inline
cols,taus = ['G90','G180','G360'], [90,180,360]
apple = problem2(cols, taus)

fig, ax = plt.subplots(figsize=(7.5,7.5))
plot2(ax, apple, cols, [r'$G_{30}$', r'$G_{90}$', r'$G_{180}$'], ['k','r', 'b'])
fig.tight_layout()
fig.savefig('../fig/06/G14.png')
plt.show()

In [ ]:
%matplotlib inline
cols,taus = ['G5','G90','G360'], [5,90,360]
apple = problem2(cols, taus)

fig, ax = plt.subplots(figsize=(7.5,7.5))
plot2(ax, apple, cols, [r'$G_{30}$', r'$G_{90}$', r'$G_{180}$'], ['k','r', 'b'])
fig.tight_layout()
fig.savefig('../fig/06/G13.png')
plt.show()

# 4

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
pltparams = {'legend.fontsize': 16, 'axes.labelsize': 20, 'axes.titlesize': 20,
             'xtick.labelsize': 12, 'ytick.labelsize':12, 'figure.figsize': (7.5, 7.5),}
plt.rcParams.update(pltparams)
def problem4(tau1, tau2):
    # Logarithmic return of price time series
    def logreturn(Pt,tau=1):
        return np.log(Pt[tau:]) - np.log(Pt[0:-tau]) # Eq.(J2) : G_tau(t) = log(S(t+tau)) - log(S(t)) 
    # Normalize data to have unit variance (<(x - <x>)^2> = 1)
    def normalized(data):
        return (data/np.sqrt(np.var(data)))

    dmy, data = np.loadtxt('../data/06/model1.txt', unpack=True)
    return normalized(logreturn(data, tau1)), normalized(logreturn(data,tau2))

def plot4(ax,g1,g16):
    def pdf(data,bins=50):
        hist,edges=np.histogram(data[~np.isnan(data)],bins=bins,density=True) # remove NaNs and compute histogram
        edges   = (edges[:-1] + edges[1:])/2.0 # get bar center
        nonzero = hist > 0.0                   # non-zero points 
        return edges[nonzero], hist[nonzero]

    for data,lbl in zip([g1,g16], [r'$G_1$', r'$G_{16}$']):
        edges, hist = pdf(np.abs(data), bins=25)
        ax.plot(edges, hist, lw=5, label=lbl)

    x = np.linspace(0, 5)
    ax.plot(x,2*np.exp(-x**2/2)/np.sqrt(2*np.pi),lw=6,ls='--',color='gray',alpha=0.6,label=r'Gaussian')
    ax.plot(x, 2*np.exp(-1.5*x),lw=6,color='k',ls='--',alpha=0.8,label=r'Exponential')
    ax.semilogy()
    ax.set_ylim([5e-4,1])
    ax.set_xlim([5e-1,6])
    ax.legend()
    ax.set_xlabel(r'Absolute normalized price return $|G_1|$')
    ax.set_ylabel(r'Probability density')


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(7.5,7.5))
g1,g2 = problem4(1,16)
plot4(ax, g1, g2)
fig.tight_layout()
fig.savefig('../fig/06/G24.png')
plt.show()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(7.5,7.5))
g1,g2 = problem4(1,2)
plot4(ax, g1, g2)
fig.tight_layout()
fig.savefig('../fig/06/G21.png')
plt.show()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(7.5,7.5))
g1,g2 = problem4(2,4)
plot4(ax, g1, g2)
fig.tight_layout()
fig.savefig('../fig/06/G22.png')
plt.show()

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(7.5,7.5))
g1,g2 = problem4(8,16)
plot4(ax, g1, g2)
fig.tight_layout()
fig.savefig('../fig/06/G23.png')
plt.show()